<a href="https://colab.research.google.com/github/threegenie/climate_classify/blob/main/climate_labeling_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
! pip install konlpy

In [2]:
%%capture
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 

fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
import os
import tqdm
import urllib.request
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import log_loss, accuracy_score,f1_score
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [5]:
# 오류(SettingWithCopyError 발생)
pd.set_option('mode.chained_assignment', 'raise') # SettingWithCopyError

# 경고(SettingWithCopyWarning 발생, 기본 값입니다)
pd.set_option('mode.chained_assignment', 'warn') # SettingWithCopyWarning

# 무시
pd.set_option('mode.chained_assignment',  None) # <==== 경고를 끈다

#### 데이터 업로드

In [6]:
import os
from google.colab import drive

drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/open/train.csv')
test = pd.read_csv('/content/drive/My Drive/open/test.csv')
label = pd.read_csv('/content/drive/My Drive/open/labels_mapping.csv')

Mounted at /content/drive


In [7]:
df.head()

,index,제출년도,사업명,사업_부처명,계속과제여부,내역사업명,과제명,요약문_연구목표,요약문_연구내용,요약문_기대효과,요약문_한글키워드,요약문_영문키워드,label
0,0,2016,농업기초기반연구,농촌진흥청,신규,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,○ 새로운 해충분류군의 동정기술 개발 및 유입확산 추적,(가) 외래 및 돌발해충의 발생조사 및 종 동정\n\n\n ○ 대상해충 : 최...,"○ 새로운 돌발 및 외래해충의 신속, 정확한 동정법 향상\n\n\n○ 돌발 및 외래...","뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체","nucleotide sequence, molecular marker, species...",24
1,1,2019,이공학학술연구기반구축(R&D),교육부,신규,지역대학우수과학자지원사업(1년~5년),대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축...,최종목표: TRAIL 감수성 표적 유전자를 발굴하고 내성제어 기전을 연구. 발굴된...,1차년도\n1) Microarray를 통한 선천적 TRAIL 내성 표적 후보 유전자...,"1) TRAIL 내성 특이적 표적분자를 발굴하고, 이를 이용한 TRAIL 효과 증진...","대장암,항암제 내성,세포사멸,유전자발굴","TRAIL,Colorectal cancer,TRAIL resistance,Apopt...",0
2,2,2016,중소기업기술혁신개발,중소기업청,신규,혁신기업기술개발,비목질계 셀룰로오스 식물자원을 활용한 기능성 부직포 및 고부가가치 뷰티케어 faci...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n* 소비자 및 바...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n- Tencel/...,* 국내 독자적인 비목질계 셀룰로오스 자원의 파이버 및 부직포 제조 등의 기술 확보...,"기능성 샐룰로오스 파이버,천연섬유,기능성 부직포,뷰티&amp;케어 제품,미용 솜","functional cellulose fiber,natural fiber,funct...",0
3,3,2018,창업성장기술개발(R&D),중소벤처기업부,신규,창업사업화연계과제,소화기 암 진단용 분자영상 형광프로브 개발,# 암특이적 바이오마커 발굴 및 바이오마커에 대한 프로브 개발\n\n\n1) 소화...,# 소화기 암 진단용 분자영상 형광프로브 개발\n\n\n- 국소 도포형 소화기 암 ...,# 암 진단기술의 차별성: 소화기 암 특이 프로브 개발\n\n\n- 최근 체외진단시...,"분자 진단,형광 조영제,프로브,항체,대장암","Molecular diagnosis,Fluorescence,probe,antibod...",0
4,4,2016,이공학개인기초연구지원,교육부,계속,기본연구지원,위암환자의 항암제반응예측을 위한 BRCA 발현검사,"수술이 불가능한 위암환자는 생존기간은 10개월 안팎에 지나지 않고, 항암화학요법에 ...",-In situ hybridization 검사의 정확성을 확인하기 위해 위암세포주 ...,-본 연구는 파라핀보관조직에서 in situ hybridization로 mRNA 및...,"BRCA,제자리부합법,조직미세배열,마이크로RNA,위암,항암제반응,젊은 연령/가족성 위암","BRCA,Insituhybridization,tissuemicroarray,micr...",0


In [8]:
label.head()

,label,소분류
0,0,00. NaN
1,1,01. 원자력 발전
2,2,02. 핵융합 발전
3,3,03. 청정화력발전·효율화
4,4,04. 수력


In [9]:
# 결측값 처리
df = df.fillna('내용없음')
test = test.fillna('내용없음')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174304 entries, 0 to 174303
Data columns (total 13 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   index      174304 non-null  int64 
 1   제출년도       174304 non-null  int64 
 2   사업명        174304 non-null  object
 3   사업_부처명     174304 non-null  object
 4   계속과제여부     174304 non-null  object
 5   내역사업명      174304 non-null  object
 6   과제명        174304 non-null  object
 7   요약문_연구목표   174304 non-null  object
 8   요약문_연구내용   174304 non-null  object
 9   요약문_기대효과   174304 non-null  object
 10  요약문_한글키워드  174304 non-null  object
 11  요약문_영문키워드  174304 non-null  object
 12  label      174304 non-null  int64 
dtypes: int64(3), object(10)
memory usage: 17.3+ MB


In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43576 entries, 0 to 43575
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      43576 non-null  int64 
 1   제출년도       43576 non-null  int64 
 2   사업명        43576 non-null  object
 3   사업_부처명     43576 non-null  object
 4   계속과제여부     43576 non-null  object
 5   내역사업명      43576 non-null  object
 6   과제명        43576 non-null  object
 7   요약문_연구목표   43576 non-null  object
 8   요약문_연구내용   43576 non-null  object
 9   요약문_기대효과   43576 non-null  object
 10  요약문_한글키워드  43576 non-null  object
 11  요약문_영문키워드  43576 non-null  object
dtypes: int64(2), object(10)
memory usage: 4.0+ MB


#### 필요한 특성만 남기기

In [12]:
df = df[['index', '요약문_연구목표','요약문_연구내용','요약문_한글키워드']]

In [13]:
test = test[['index', '요약문_연구목표','요약문_연구내용','요약문_한글키워드']]

#### 데이터 정제 - 훈련, 테스트셋에 모두 적용 -> 정규표현식 사용하여 한글 제외 모두 제거

In [14]:
df['요약문_연구목표'] = df['요약문_연구목표'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['요약문_연구목표'].replace('', np.nan, inplace=True)

df['요약문_연구내용'] = df['요약문_연구내용'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['요약문_연구내용'].replace('', np.nan, inplace=True)

df['요약문_한글키워드'] = df['요약문_한글키워드'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['요약문_한글키워드'].replace('', np.nan, inplace=True)

In [15]:
test['요약문_연구목표'] = test['요약문_연구목표'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test['요약문_연구목표'].replace('', np.nan, inplace=True)

test['요약문_연구내용'] = test['요약문_연구내용'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test['요약문_연구내용'].replace('', np.nan, inplace=True)

test['요약문_한글키워드'] = test['요약문_한글키워드'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test['요약문_한글키워드'].replace('', np.nan, inplace=True)

In [16]:
df.head()

,index,요약문_연구목표,요약문_연구내용,요약문_한글키워드
0,0,새로운 해충분류군의 동정기술 개발 및 유입확산 추적,가 외래 및 돌발해충의 발생조사 및 종 동정 대상해충 최근 새롭게 발견된 ...,뉴클레오티드 염기서열 분자마커 종 동정 침샘 전사체
1,1,최종목표 감수성 표적 유전자를 발굴하고 내성제어 기전을 연구 발굴된 유전자를 통...,차년도 를 통한 선천적 내성 표적 후보 유전자 선별 대장암 환자조직을 이용하여 후...,대장암항암제 내성세포사멸유전자발굴
2,2,식물계자원 정련 및 최적 신서란 파이버 기초연구 개발 소비자 및 바이어들의 피드백...,식물계자원 정련 및 최적 신서란 파이버 기초연구 개발 파이버 기초 물성 연구 및...,기능성 샐룰로오스 파이버천연섬유기능성 부직포뷰티케어 제품미용 솜
3,3,암특이적 바이오마커 발굴 및 바이오마커에 대한 프로브 개발 소화기 암 진단용 분...,소화기 암 진단용 분자영상 형광프로브 개발 국소 도포형 소화기 암 분자 영상 프로...,분자 진단형광 조영제프로브항체대장암
4,4,수술이 불가능한 위암환자는 생존기간은 개월 안팎에 지나지 않고 항암화학요법에 의한 ...,검사의 정확성을 확인하기 위해 위암세포주 를 제작하고 발현을 결과를 ...,제자리부합법조직미세배열마이크로위암항암제반응젊은 연령가족성 위암


In [17]:
test.head()

,index,요약문_연구목표,요약문_연구내용,요약문_한글키워드
0,174304,차체 점용접부의 품질 검사를 위한 채널 무선 기반 탐촉자 개발 모바일 단말기를 ...,차년도 개발 탐촉 시스템의 성능 평가 위한 표준 시편 제작 시스템 개발 ...,마찰교반점용접 비파괴 검사 초음파 탐상 씨 스캔 용접 품질 평가
1,174305,자연계에는 입자의 개수가 아주 큰 다양한 다입자계가 존재한다 이런 다입자계의 효율...,연구과제 무한입자계의 동역학 작용소 방정식에 대한 연구 하이젠베르크 그림에서의...,다체계 방정식동역학의 안정성양자역학고전역학평균장 극한고전극한비상대론적 극한
2,174306,저출생체중아는 정상 체중 출생아에 비하여 운동 정신 행동 및 정서발달장애가 빈번하게...,본 연구는 퇴원 후 저출생체중아의 퇴원 후 추후관리 프로그램 중 가정방문 모델과 가...,저출생체중아추후관리조기개입자조모임가정방문
3,174307,최근 초정밀 공작기계 시장의 트렌드를 살펴보면 다음과 같다 자동차 항공 의료 광학 ...,의 가공정밀도면조도 구현 국내에서는 연삭기를 제외하고 머시닝센터 등의 공작기계에...,초정밀가공폴리싱 가공마이크로경면가공난삭재가공
4,174308,본 연구의 최종목표는 다결정 금속재료의 미세조직구조를 고려한 확률론적 응력부식균열 ...,총 연구기간은 년으로 연차별 연구내용 및 범위는 다음과 같다 차년도 단결...,결정구조응력부식균열확률론적모델


#### 토큰화 

In [18]:
from konlpy.tag import Okt
okt = Okt()

In [19]:
from konlpy.tag import Komoran
komoran = Komoran()

In [20]:
stop_words=['은','는','이','가', '하','아','것','들','의','있','되','수','보','주','등','한']

In [21]:
df['연구목표_T'] = None
df['연구내용_T'] = None
df['한글키워드_T'] = None

In [22]:
test['연구목표_T'] = None
test['연구내용_T'] = None
test['한글키워드_T'] = None

In [23]:
def preprocessing(text, okt, remove_stopwords=False, stop_words=[]):
    word_text=okt.morphs(text, stem=True)
    if remove_stopwords:
        word_review=[token for token in word_text if not token in stop_words]
    return word_review

In [ ]:
for text in tqdm.tqdm(df['요약문_연구목표']):
    try:
        df['연구목표_T'].append(preprocessing(text, okt, remove_stopwords=True, stop_words=stop_words))
    except:
        df['연구목표_T'].append([])

 39%|███▉      | 68657/174304 [17:51<25:38, 68.65it/s]

In [ ]:
for text in tqdm.tqdm(df['요약문_연구내용']):
    try:
        df['연구내용_T'].append(preprocessing(text, okt, remove_stopwords=True, stop_words=stop_words))
    except:
        df['연구내용_T'].append([])

In [ ]:
for text in tqdm.tqdm(df['요약문_한글키워드']):
    try:
        df['한글키워드_T'].append(preprocessing(text, okt, remove_stopwords=True, stop_words=stop_words))
    except:
        df['한글키워드_T'].append([])

In [ ]:
for text in tqdm.tqdm(test['요약문_연구목표']):
    try:
        test['연구목표_T'].append(preprocessing(text, okt, remove_stopwords=True, stop_words=stop_words))
    except:
        test['연구목표_T'].append([])

In [ ]:
for text in tqdm.tqdm(test['요약문_연구내용']):
    try:
        test['연구내용_T'].append(preprocessing(text, okt, remove_stopwords=True, stop_words=stop_words))
    except:
        test['연구내용_T'].append([])

In [ ]:
for text in tqdm.tqdm(test['요약문_한글키워드']):
    try:
        test['한글키워드_T'].append(preprocessing(text, okt, remove_stopwords=True, stop_words=stop_words))
    except:
        test['한글키워드_T'].append([])

In [ ]:
# df['연구목표_T'] = df['요약문_연구목표'].apply(okt.morphs)
# df['연구목표_T'] = df['요약문_연구목표'].apply(lambda x: [item for item in x if item not in stop_words])
# df['연구내용_T'] = df['요약문_연구내용'].apply(okt.morphs)
# df['연구내용_T'] = df['요약문_연구내용'].apply(lambda x: [item for item in x if item not in stop_words])
# df['한글키워드_T'] = df['요약문_한글키워드'].apply(okt.morphs)
# df['한글키워드_T'] = df['요약문_한글키워드'].apply(lambda x: [item for item in x if item not in stop_words])

In [ ]:
# test['연구목표_T'] = test['요약문_연구목표'].apply(okt.morphs)
# test['연구목표_T'] = test['요약문_연구목표'].apply(lambda x: [item for item in x if item not in stop_words])
# test['연구내용_T'] = test['요약문_연구내용'].apply(okt.morphs)
# test['연구내용_T'] = test['요약문_연구내용'].apply(lambda x: [item for item in x if item not in stop_words])
# test['한글키워드_T'] = test['요약문_한글키워드'].apply(okt.morphs)
# test['한글키워드_T'] = test['요약문_한글키워드'].apply(lambda x: [item for item in x if item not in stop_words])

In [ ]:
df.head(10)

In [ ]:
test.head(10)